## 🚀 Day2-02：Agent Tools 模式与最佳实践

5 天 AI Agents 强化课程的第二天，我们将学习如何使用工具来扩展智能体的功能。

上小节我们学习了如何将自定义 Python 函数作为工具添加到您的智能体中。
现在，我们将更进一步：使用外部 MCP 服务并处理长时间运行的操作。

你讲学习到内容：

- ✅ 连接外部 MCP 服务器
- ✅ 实现可暂停代理执行以等待外部输入的长时运行操作
- ✅ 构建可恢复的工作流，在对话中断时保持状态
- ✅ 了解何时以及如何使用这些模式

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

入门步骤：

1.vscode 安装 jupyter 插件

# Section 1：设置环境

##### 1.1 配置你的 Gemini API密钥

In [ ]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key setup complete.")
            break
else:
    print(f"⚠️ Please create .env file at: {env_file}")


✅ Gemini API key setup complete.


##### 1.2 导入 ADK 组件

基于 Google 的 Agent Development Kit (ADK) 框架来构建 Agent。ADK 提供了一组工具和库，用于构建和运行 Agent。

In [ ]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


##### 1.3：配置重试选项

当你在使用LLM时，可能会遇到瞬时错误，如速率限制或临时服务不可用。重试选项通过指数退避自动重试请求来处理这些故障。

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# 🤖 Section 2: MCP（Model Context Protocol）

在第一小节，我们已经学会了如何为代理创建自定义函数。

但连接到外部系统（如GitHub、数据库、Slack）需要编写和维护API客户端，那样对小白门槛太高了。

解决方案是：使用模型上下文协议（MCP），是一个开放标准，允许代理使用社区构建的集成，无需编写自己的集成和API客户端，只需连接到现有的MCP服务器即可。

MCP 使 Agent 能够做的事情：

✅ 无需自定义集成代码即可访问数据库、API 和服务中的实时外部数据

✅ 利用具有标准化接口的社区构建工具

✅ 通过连接多个专用服务器扩展功能

##### 2.1 MCP 的工作原理

MCP 将你的 Agents（客户端）连接到提供工具的外部 MCP 服务器：

• MCP服务器：提供特定工具（如图像生成、数据库访问、Figma UI）

• MCP客户端： 你的 Agent ，使用这些工具

• 所有服务器以相同方式工作——标准化接口

MCP 架构原理 https://cloud.google.com/discover/what-is-model-context-protocol?hl=zh-CN#mcp-architecture-and-components

![MCP架构图](https://fisherai-1312281807.cos.ap-guangzhou.myqcloud.com/202511202317080.png)

##### 2.2 在 Agent 中使用 MCP

使用 MCP 只需四个步骤：

1. **选择 MCP 服务器** - 本次使用 Everything Server（测试用）
2. **创建 MCP 工具集** - 配置连接参数
3. **添加到 Agent** - 将工具集加入 Agent
4. **运行测试** - 验证功能

**步骤 1：选择 MCP 服务器**

本次使用 **Everything Server**（`@modelcontextprotocol/server-everything`）- 一个测试用的 MCP 服务器，提供 `getTinyImage` 工具来生成 16x16 像素的测试图像。

**步骤 2：创建 MCP 工具集**

配置连接参数，使用 npx 运行 MCP 服务器：

In [ ]:
# 创建 MCP 工具集：连接到 Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # 通过 npx 运行 MCP 服务器
            args=[
                "-y",  # npx 参数：自动确认安装
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],  # 只使用 getTinyImage 工具
        ),
        timeout=30,  # 连接超时时间：30秒
    )
)

print("✅ MCP 工具集创建成功")

✅ MCP 工具集创建成功


**🔍 幕后工作原理：**

当你运行上面的代码时，ADK 会自动完成以下步骤：

1. **启动服务器**：运行 `npx -y @modelcontextprotocol/server-everything`
2. **建立连接**：通过 stdio 建立通信通道
3. **工具发现**：服务器告诉 ADK 可用的工具
4. **自动集成**：工具自动添加到 Agent 的工具列表中

**步骤 3：创建 Agent 并添加 MCP 工具**

In [ ]:
# 创建集成了 MCP 工具的 Agent
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.0-flash-exp", retry_options=retry_config),
    name="image_agent",
    instruction="使用 MCP 工具为用户生成图像",
    tools=[mcp_image_server],
)

# 创建 Session Service 和 Runner
session_service = InMemorySessionService()
runner = Runner(
    agent=image_agent,
    session_service=session_service
)

print("✅ Agent 和 Runner 创建成功")

NameError: name 'InMemoryRunner' is not defined

**步骤 4：运行测试**

In [ ]:
# 创建会话并运行 Agent
session_id = str(uuid.uuid4())

# 运行 Agent 并请求生成图像
response = await runner.run(
    session_id=session_id,
    new_message="请提供一张示例的微型图像"
)

print("\n📝 Agent 响应：")
for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, 'text'):
                print(part.text)

解码并显示返回的图像：

In [ ]:
from IPython.display import display, Image as IPImage
import base64

# 从响应中提取并显示图像
for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        # 解码 Base64 数据并显示图像
                        display(IPImage(data=base64.b64decode(item["data"])))

# 🔄 Section 3: 长时运行操作与可恢复工作流

在实际应用中，Agent 经常需要执行需要等待外部输入的操作，例如：

- 等待用户批准
- 等待外部系统响应
- 等待人工审核

ADK 提供了 **可恢复性（Resumability）** 功能来处理这些场景。

##### 3.1 什么是可恢复性？

可恢复性允许 Agent：

1. **暂停执行** - 在需要外部输入时暂停
2. **保存状态** - 保存当前对话和上下文
3. **恢复执行** - 收到输入后从暂停点继续

这对于需要人工干预或长时间等待的工作流至关重要。

##### 3.2 创建需要批准的工具

让我们创建一个模拟需要人工批准的工具：

In [ ]:
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.tool_context import ToolContext

def request_approval(action: str, tool_context: ToolContext) -> str:
    """
    请求用户批准某个操作
    
    Args:
        action: 需要批准的操作描述
        tool_context: 工具上下文，用于暂停执行
    
    Returns:
        批准状态消息
    """
    # 暂停 Agent 执行，等待用户输入
    tool_context.pause_agent(
        message=f"请批准以下操作：{action}\n请回复 'approved' 或 'rejected'"
    )
    return f"等待批准：{action}"

# 创建工具
approval_tool = FunctionTool(func=request_approval)

print("✅ 批准工具创建成功")

##### 3.3 创建支持可恢复性的 Agent

使用 `App` 类来创建支持可恢复性的 Agent：

In [ ]:
# 创建支持可恢复性的 Agent
approval_agent = LlmAgent(
    model=Gemini(model="gemini-2.0-flash-exp", retry_options=retry_config),
    name="approval_agent",
    instruction="你是一个需要用户批准才能执行操作的助手。在执行重要操作前，使用 request_approval 工具请求批准。",
    tools=[approval_tool],
)

# 创建 App 并启用可恢复性
app = App(
    agent=approval_agent,
    resumability_config=ResumabilityConfig(enabled=True)
)

print("✅ 可恢复 Agent 创建成功")

##### 3.4 测试可恢复工作流

让我们测试暂停和恢复功能：

In [ ]:
# 创建新会话
session_id = str(uuid.uuid4())

# 启动需要批准的工作流
print("🚀 启动工作流...")
response = await app.run(
    session_id=session_id,
    new_message="请帮我删除所有旧文件"
)

# 显示响应
for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, 'text'):
                print(f"\n📝 Agent: {part.text}")

print(f"\n💾 会话 ID: {session_id}")
print("⏸️  工作流已暂停，等待批准...")

##### 3.5 恢复工作流

现在让我们提供批准并恢复执行：

In [ ]:
# 恢复工作流并提供批准
print("\n✅ 提供批准并恢复工作流...")
response = await app.run(
    session_id=session_id,
    new_message="approved"
)

# 显示最终响应
for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, 'text'):
                print(f"\n📝 Agent: {part.text}")

print("\n🎉 工作流完成！")

# 📚 Section 4: 最佳实践总结

##### 4.1 何时使用 MCP？

✅ **适合使用 MCP 的场景：**
- 需要连接外部服务（GitHub、Slack、数据库等）
- 想要使用社区构建的工具
- 需要标准化的工具接口
- 希望快速集成而不编写 API 客户端

❌ **不适合使用 MCP 的场景：**
- 简单的内部函数（使用 FunctionTool 更简单）
- 需要高度定制的逻辑
- 性能要求极高的场景

##### 4.2 何时使用可恢复性？

✅ **适合使用可恢复性的场景：**
- 需要人工批准或审核
- 等待外部系统响应（支付、审批流程等）
- 长时间运行的工作流
- 需要在多个会话间保持状态

❌ **不需要可恢复性的场景：**
- 简单的问答对话
- 一次性完成的任务
- 不需要外部输入的操作

##### 4.3 关键要点

1. **MCP 简化集成** - 无需编写 API 客户端，直接使用标准化工具
2. **可恢复性处理中断** - 优雅地处理需要等待的操作
3. **选择合适的模式** - 根据需求选择 FunctionTool、MCP 或可恢复性
4. **保持简单** - 从简单开始，需要时再添加复杂功能

# 🎯 下一步

恭喜！你已经学会了：

✅ 使用 MCP 连接外部服务
✅ 实现长时运行操作
✅ 构建可恢复的工作流
✅ 了解何时使用不同的模式

**继续学习：**
- Day 3: Agent 会话管理和记忆
- Day 4: Agent 可观测性
- Day 5: Agent 间通信

**实践建议：**
1. 尝试连接其他 MCP 服务器（如 GitHub、Slack）
2. 构建需要多步批准的工作流
3. 结合 MCP 和可恢复性创建复杂应用